<a href="https://colab.research.google.com/github/Vasilevykh-M/BoxersAtari/blob/main/boxers_atari.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Примонтирование Google Drive
# МЕНЯ ВСТАВИТЬ В САМОЕ НАЧАЛО, А ПРИ ПОВТОРНОЕ РАБОТЕ ВСЕГДА ЗАПУСКАТЬ!
FOLDER = "agent_model" # Тут имя папки, в которую все сохраняется. Менять это имя можно.
                        #  На Google Drive должно быть та же папка
                        # Регистр (ЗАГЛАВНЫЕ или строчные буквы) важен!

PATH = "drive/MyDrive/" + FOLDER  + "/"
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append(PATH)

Mounted at /content/drive


In [ ]:
!apt-get install -y xvfb python-opengl ffmpeg x11-utils > /dev/null 2>&1

In [ ]:
!pip install pettingzoo
!pip install multi-agent-ale-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 816 kB 4.9 MB/s 
     |████████████████████████████████| 836 kB 54.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 778 kB 5.0 MB/s 


In [ ]:
!pip install autorom[accept-rom-license]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=441028 sha256=f22810670fbe5cdfaa79f389faff2e565bded81ec3d57e398fad23a35416e232
  Stored in directory: /root/.cache/pip/wheels/87/67/2e/6147e7912fe37f5408b80d07527dab807c1d25f5c403a9538a
Successfully built AutoROM.accept-rom-license


In [ ]:
# (!) Нужно будет наyжать "Y"
!AutoROM

AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	/usr/local/lib/python3.7/dist-packages/AutoROM/roms
	/usr/local/lib/python3.7/dist-packages/multi_agent_ale_py/roms

Existing ROMs will be overwritten.

I own a license to these Atari 2600 ROMs.
I agree to not distribute these ROMs and wish to proceed: [Y/n]: Y
Installed /usr/local/lib/python3.7/dist-packages/AutoROM/roms/adventure.bin
Installed /usr/local/lib/python3.7/dist-packages/multi_agent_ale_py/roms/adventure.bin
Installed /usr/local/lib/python3.7/dist-packages/AutoROM/roms/air_raid.bin
Installed /usr/local/lib/python3.7/dist-packages/multi_agent_ale_py/roms/air_raid.bin
Installed /usr/local/lib/python3.7/dist-packages/AutoROM/roms/alien.bin
Installed /usr/local/lib/python3.7/dist-packages/multi_agent_ale_py/roms/alien.bin
Installed /usr/local/lib/python3.7/dist-packages/AutoROM/roms/amidar.bin
Installed /usr/local/lib/python3.7/dist-packages/multi_agent_ale_py/roms/amidar.bin
Installed /usr/local/lib/python

In [ ]:
from IPython.display import clear_output, HTML
from IPython import display as ipythondisplay
import base64
import os
import io

def show_video(image_path):

  if not os.path.exists(f'{image_path}.mp4'):
    os.system(f"ffmpeg -framerate 60 -pattern_type glob -i '{image_path}/*.png' -c:v libx264 -pix_fmt yuv420p {image_path}.mp4")

  video = io.open(f'{image_path}.mp4', 'r+b').read()
  encoded = base64.b64encode(video)
  ipythondisplay.display(HTML(data='''<video alt="test" autoplay
          controls style="height: 400px;">
          <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        </video>'''.format(encoded.decode('ascii'))))


In [ ]:
from pettingzoo.atari import boxing_v2

In [ ]:
env = boxing_v2.parallel_env()

In [ ]:
env.observation_spaces

/usr/local/lib/python3.7/dist-packages/pettingzoo/utils/conversions.py:92: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
  "The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead."


{'first_0': Box(0, 255, (210, 160, 3), uint8),
 'second_0': Box(0, 255, (210, 160, 3), uint8)}

In [ ]:
env.action_spaces

/usr/local/lib/python3.7/dist-packages/pettingzoo/utils/conversions.py:106: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
  "The `action_spaces` dictionary is deprecated. Use the `action_space` function instead."


{'first_0': Discrete(18), 'second_0': Discrete(18)}

In [ ]:
import numpy as np

def invert_players(image):
  img = image[25:-25,25:-25,:]
  flags = img.sum(axis=2)
  img[flags == 0] = np.array([214]*3)
  img[flags == 214*3] = np.array([0,0,0])
  image[25:-25,25:-25,:] = img
  return image

def correct_observations(observations):
  observations["second_0"] = invert_players(observations["second_0"].copy())
  return observations

In [ ]:
def get_boxer_coordinate(image, player):
  mean_im = (image[25:-25, 25:-25, 0] + image[25:-25, 25:-25, 1] + image[25:-25, 25:-25, 2]) / 3.0
  if player == "first_0":
    mean_pos = (mean_im > 210.0).nonzero()
    return torch.round(mean_pos[0].float().mean()), torch.round(mean_pos[1].float().mean())
  else:
    mean_pos = (mean_im < 30.0).nonzero()
    return torch.round(mean_pos[0].float().mean()), torch.round(mean_pos[1].float().mean())

In [ ]:
import torch
from torch.distributions import Categorical
from torchvision.utils import save_image
import os

def play_with_agents(env, agents, max_steps = 200, device = None, render = True):
  if device:
    for name, agent in agents.items():
      agent.to(device)

  if render:
    os.system("mkdir -p images")
    os.system("rm -r images/*")

  frame = 0

  total_reward = {}
  render_result = {}
  trajectories = {agent_name:{'observations':[], 'actions':[], 'rewards':[]} for agent_name in agents.keys()} # Заполнение этого словаря -- одно из заданий

  observations = correct_observations(env.reset())

  old_r_x_w = 200.0
  old_r_y_w = 150.0
  old_r_x_b = 200.0
  old_r_y_b = 150.0

  old_ewklid_w = 250.0
  old_ewklid_b = 250.0
  epoch = 0


  while env.agents:
    if render:
      # Сохраняем картинки для всех агентов
      for a in env.agents:
        agent_path = f'images/{a}/'
        if a not in render_result:
          render_result[a] = agent_path
          os.system(f"mkdir -p images/{a}")

        img = torch.tensor(observations[a],dtype=torch.float64)
        img /= 255.0
        img = img.permute(2,0,1)
        save_image(img, f'{agent_path}/frame{frame:05d}.png')

    frame += 1

    actions = {}
    for agent_name in env.agents:
      probs = agents[agent_name](
          torch.FloatTensor(observations[agent_name]).to(device))

      dist = Categorical(probs)
      action = dist.sample().item()

      actions[agent_name] = action

      trajectories[agent_name]['observations'].append(observations[agent_name])
      trajectories[agent_name]['actions'].append(action)


    observations, rewards, terminations, truncations, infos = env.step(actions)
    observations = correct_observations(observations)

    for agent_name in agents:

      reward_step = 0.0
      reward_step_x = 0.0
      reward_step_y = 0.0
      reward_ewklid = 0.0

      if agent_name == 'first_0':

        x_w, y_w = get_boxer_coordinate(torch.FloatTensor(observations['first_0']), 'first_0')
        x_b, y_b = get_boxer_coordinate(torch.FloatTensor(observations['first_0']), 'second_0')

        manhat_x = abs(x_w - x_b)
        manhat_y = abs(y_w - y_b)
        ewklid = (manhat_x**2 + manhat_y**2)**0.5

        reward_step_x = (0.5 if old_r_x_w > manhat_x > 10.0 else -1.0)
        reward_step_y = (0.5 if old_r_y_w > manhat_y > 10.0 else -1.0)

        old_r_x_w = manhat_x
        old_r_y_w = manhat_y

      else:

        x_w, y_w = get_boxer_coordinate(torch.FloatTensor(observations['second_0']), 'second_0')
        x_b, y_b = get_boxer_coordinate(torch.FloatTensor(observations['second_0']), 'first_0')

        manhat_x = abs(x_w - x_b)
        manhat_y = abs(y_w - y_b)
        ewklid = (manhat_x**2 + manhat_y**2)**0.5

        reward_step_x = (0.5 if old_r_x_w > manhat_x > 10.0 else -1.0)
        reward_step_y = (0.5 if old_r_y_w > manhat_y > 10.0 else -1.0)


        old_r_x_b = manhat_x
        old_r_y_b = manhat_y



      if reward_step_x > 0:
        reward_step = reward_step_x
      else:
        if reward_step_y > 0:
          reward_step = reward_step_y
        else:
          reward_step = reward_step_x



      total_reward[agent_name] = total_reward.get(agent_name, 0) + rewards[agent_name] + reward_step
      trajectories[agent_name]['rewards'].append(rewards[agent_name]  + reward_step)

    epoch+=1


    if all([f for f in terminations.values()]):
      break
    if frame > max_steps:
      break

  if render:
    #create_video("images/")
    os.system("ffmpeg -framerate 60 -pattern_type glob -i './images/*.png' -c:v libx264 -pix_fmt yuv420p out.mp4")

  return trajectories, total_reward, render_result


In [ ]:
def compute_total_reward(trajectory, discount = 0.9):
  R = np.array(trajectory["rewards"])
  for i in reversed(range(R.shape[0]-1)):
    R[i] += R[i+1] * discount
  trajectory['R'] = R
  return R

In [ ]:
def compute_trajectory_loss(trajectory, agent):
  loss = 0
  for i in range(len(trajectory['observations'])):
    p = agent(torch.FloatTensor(trajectory['observations'][i]))
    p_a = p[trajectory['actions'][i]]
    loss += -trajectory['R'][i] * torch.log(p_a + 1e-18)
  return loss

In [ ]:
# Основной цикл обучения
NUM_EPISODES = 5 # количество эпизодов, которые будут сыграны для обучения
DISCOUNT = 0.99 # фактор дисконтирования
TRAIN_EVERY = 1 # Сколько нужно накопить траекторий с одной стратегией прежде чем обновлять веса

In [ ]:
class ExampleAgent(torch.nn.Module):
    def __init__(self, device):
        super(ExampleAgent, self).__init__()
        self.device = device
        self.l1 = torch.nn.Linear(2, 40)
        self.l2 = torch.nn.Linear(40, 40000)
        self.ReLU = torch.nn.LeakyReLU()
        self.l3 = torch.nn.Linear(40000, 5)
        self.softmax = torch.nn.Softmax()

    def get_boxer_coordinate(self, image, player):
        mean_im = (image[25:-25, 25:-25, 0] + image[25:-25, 25:-25, 1] + image[25:-25, 25:-25, 2]) / 3.0
        if player == "first_0":
          mean_pos = (mean_im > 210.0).nonzero()
          return mean_pos[0].float().min(), mean_pos[0].float().max(), mean_pos[1].float().min(), mean_pos[1].float().max()
        else:
          mean_pos = (mean_im < 30.0).nonzero()
          return mean_pos[0].float().min(), mean_pos[0].float().max(), mean_pos[1].float().min(), mean_pos[1].float().max()

    def __call__(self, state):

      image = state.to(self.device)

      x_w_1, x_w_2, y_w_1, y_w_2 = self.get_boxer_coordinate(image, 'first_0')
      x_b_1, x_b_2, y_b_1, y_w_2 = self.get_boxer_coordinate(image, 'second_0')
      d_x = ((x_w_1-x_b_1) - (x_w_2 - x_b_2)) / 200.0
      d_y = ((y_w_1 - y_b_1) - (y_w_2 - y_w_2)) / 300.0

      data1 = torch.tensor([d_x, d_y]).to(self.device)

      data = self.ReLU(self.l1(data1))
      data = self.l2(data)
      data = self.l3(data)
      data = torch.cat((torch.zeros(1).to(self.device), data, torch.zeros(12).to(self.device)))

      return self.softmax(data)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from torch.optim import AdamW

agent_first = ExampleAgent(device)
agent_first.to(device)
agent_first.load_state_dict(torch.load('my_agent.pth'))
#agent_first.load_state_dict(torch.load('state_dict_5.pth'))#, map_location=torch.device('cpu')))

second_agent = ExampleAgent(device)
second_agent.to(device)
#second_agent.load_state_dict(torch.load('my_agent.pth'))
#second_agent.load_state_dict(torch.load('state_dict_5.pth'))#, map_location=torch.device('cpu')))

ExampleAgent(
  (l1): Linear(in_features=2, out_features=40, bias=True)
  (l2): Linear(in_features=40, out_features=40000, bias=True)
  (ReLU): LeakyReLU(negative_slope=0.01)
  (l3): Linear(in_features=40000, out_features=5, bias=True)
  (softmax): Softmax(dim=None)
)

# Учим первого агента

In [ ]:
agent_first.train()
agent_first.to(device)

second_agent.eval()
second_agent.to(device)

optim_first = AdamW(agent_first.parameters(), lr = 1e-3)

trajectories = []
cur_total_reward = 0

for game in range(NUM_EPISODES):

  trajs, total_rewards, _ = play_with_agents(
    env,
    {'first_0': agent_first, 'second_0': second_agent},
    max_steps = 600, device = device, render = False)

  # Получаем интересующую нас траекторию
  traj = trajs['first_0']

  cur_total_reward += total_rewards['first_0']
  # Расчитываем накопительную награду для этой траектории (колонка R)
  compute_total_reward(trajs['first_0'], discount = 0.99)
  # Сохраняем траекторию

  trajectories.append(traj)

  if (game % TRAIN_EVERY) == 0:
    # Теперь непосредственно обучение.
    loss = 0
    for traj in trajectories:
      loss += compute_trajectory_loss(traj, agent_first)

    print(f'Trained GAME {game}. Avg. Total Reward is {cur_total_reward / TRAIN_EVERY} loss {loss.item()}.')

    optim_first.zero_grad()
    loss.backward()
    optim_first.step()

    trajectories = []
    cur_total_reward = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Trained GAME 0. Avg. Total Reward is -242.5 loss -52725.66015625.
Trained GAME 1. Avg. Total Reward is -257.5 loss -57513.0078125.
Trained GAME 2. Avg. Total Reward is -257.5 loss -57350.62109375.
Trained GAME 3. Avg. Total Reward is -284.5 loss -63058.65234375.
Trained GAME 4. Avg. Total Reward is -203.5 loss -47051.40625.


In [ ]:
# И играем нашим агентом...
# Играем агентами
with torch.no_grad():
  agent_first.eval()
  _, rewards, renders = play_with_agents( env,
                  {'first_0': agent_first, 'second_0': second_agent },
                  max_steps = 1000,
                  render = True)
rewards

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


{'first_0': -522.5, 'second_0': -1001.0}

In [ ]:
show_video(renders['first_0'])

# Учим второго агента

In [ ]:
agent_first.train()
agent_first.to(device)

second_agent.load_state_dict(torch.load('my_agent.pth'))
second_agent.eval()
second_agent.to(device)

trajectories = []
cur_total_reward = 0

for game in range(NUM_EPISODES):

  trajs, total_rewards, _ = play_with_agents(
    env,
    {'first_0': second_agent, 'second_0': agent_first},
    max_steps = 600, device = device, render = False)

  # Получаем интересующую нас траекторию
  traj = trajs['second_0']

  cur_total_reward += total_rewards['second_0']
  # Расчитываем накопительную награду для этой траектории (колонка R)
  compute_total_reward(trajs['second_0'], discount = 0.99)
  # Сохраняем траекторию

  trajectories.append(traj)

  if (game % TRAIN_EVERY) == 0:
    # Теперь непосредственно обучение.
    loss = 0
    for traj in trajectories:
      loss += compute_trajectory_loss(traj, agent_first)

    print(f'Trained GAME {game}. Avg. Total Reward is {cur_total_reward / TRAIN_EVERY} loss {loss.item()}.')

    optim_first.zero_grad()
    loss.backward()
    optim_first.step()

    trajectories = []
    cur_total_reward = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Trained GAME 0. Avg. Total Reward is -268.0 loss -61229.1640625.
Trained GAME 1. Avg. Total Reward is -301.0 loss -60506.7578125.
Trained GAME 2. Avg. Total Reward is -268.0 loss -60714.35546875.
Trained GAME 3. Avg. Total Reward is -331.0 loss -45327.15234375.
Trained GAME 4. Avg. Total Reward is -386.5 loss -3180.967529296875.


In [ ]:
# И играем нашим агентом...
# Играем агентами
with torch.no_grad():
  agent_first.eval()
  _, rewards, renders = play_with_agents( env,
                  {'first_0': second_agent, 'second_0': agent_first},
                  max_steps = 1000,
                  render = True)
rewards

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


{'first_0': -650.0, 'second_0': -650.0}

In [ ]:
show_video(renders['first_0'])

In [ ]:
torch.save(agent_first.state_dict(), "my_agent.pth")